In [5]:
import pandas as pd
import numpy as np
import lightgbm as lgb
dataset_dir = '../Data/label.txt'
org = np.loadtxt(dataset_dir)
# org = pd.read_csv(dataset_dir, sep=' ',dtype={'balance':np.float64}, header=None)

In [1]:
import platform 
platform.architecture()

('64bit', 'ELF')

In [6]:
org.shape

(371751, 60)

In [7]:
test_set = org[-10:, :]

In [8]:
misscolidx = 6
test_set_pre = np.delete(test_set, misscolidx, axis=1)
# org_train = np.delete(org[:10000], misscolidx, axis=1)

In [4]:

def LightGBM(X, y):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'learning_rate': 0.015,
        'cat_smooth': 10,
        'feature_fraction': 0.8,
        'bagging_freq': 5,
        'verbosity': 0
    }
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=70)
    # 用得到的模型进行未知年龄结果预测
    predicted_feature = gbm.predict(test_X, num_iteration=gbm.best_iteration)
    print("---------best_iteration: ", gbm.best_iteration)
    return predicted_feature
LightGBM()

ImportError: No module named lightgbm

ImportError: No module named 'sklearn.model_selection'

In [13]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import random
def RandomForest(x,y,rf_iteration):
    rfr = RandomForestRegressor(random_state=0, n_estimators=rf_iteration,  n_jobs=-1)
    rfr.fit(x,y)
    del x,y
    return rfr

def LightGBM(X, y, gbm_iteration):
    from sklearn.cross_validation import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    params = {
        'boosting': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'num_leaves': 31,
        'min_data_in_leaf': 20,
        'learning_rate': 0.015,
        'cat_smooth': 10,
        'feature_fraction': 0.8,
        'bagging_freq': 5,
        'verbosity': 0
    }
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=gbm_iteration,
                    valid_sets=lgb_eval,
                    early_stopping_rounds=70)
    return gbm

rand_num = 10000
rf_iteration = 2000
gbm_iteration = 1000
rand_t = random.sample(range(org.shape[0]-10), rand_num)
x = np.delete(org[rand_t],misscolidx,axis=1)
y = org[rand_t,misscolidx]

import time
ts = time.clock()
m1 = RandomForest(x,y,rf_iteration)
te = time.clock()
print("Train RandomForest done. Time:{}s".format(te-ts))

ts = time.clock()
m2 = LightGBM(x,y, gbm_iteration)
te = time.clock()
print("Train LightGBM done. Time:{}s".format(te-ts))


res1 = m1.predict(test_set_pre)
print("****** RF result ******")
print("****** data_set={}, iterations={}".format(rand_num,rf_iteration))
print(res1)
print("****** L2 dist ******")
print(np.linalg.norm(res1 - org[-10:,misscolidx]))



res2 = m2.predict(test_set_pre)
print("****** LightGBM result ******")
print("****** data_set={}, iterations={}, best_iteration".format(rand_num, gbm_iteration, m2.best_iteration))
print(res2)
print("****** L2 dist ******")
print(np.linalg.norm(res2 - org[-10:,misscolidx]))

Train RandomForest done. Time:369.754362s
[1]	valid_0's rmse: 2930.47
Training until validation scores don't improve for 70 rounds.
[2]	valid_0's rmse: 2900.92
[3]	valid_0's rmse: 2871.4
[4]	valid_0's rmse: 2842.38
[5]	valid_0's rmse: 2813.55
[6]	valid_0's rmse: 2784.97
[7]	valid_0's rmse: 2756.68
[8]	valid_0's rmse: 2728.57
[9]	valid_0's rmse: 2700.72
[10]	valid_0's rmse: 2673.13
[11]	valid_0's rmse: 2645.73
[12]	valid_0's rmse: 2618.47
[13]	valid_0's rmse: 2591.54
[14]	valid_0's rmse: 2564.54
[15]	valid_0's rmse: 2537.87
[16]	valid_0's rmse: 2511.38
[17]	valid_0's rmse: 2485.23
[18]	valid_0's rmse: 2459.17
[19]	valid_0's rmse: 2433.18
[20]	valid_0's rmse: 2407.54
[21]	valid_0's rmse: 2382.14
[22]	valid_0's rmse: 2357.1
[23]	valid_0's rmse: 2331.96
[24]	valid_0's rmse: 2307
[25]	valid_0's rmse: 2282.43
[26]	valid_0's rmse: 2257.9
[27]	valid_0's rmse: 2233.46
[28]	valid_0's rmse: 2209.28
[29]	valid_0's rmse: 2185.51
[30]	valid_0's rmse: 2163.02
[31]	valid_0's rmse: 2139.43
[32]	valid_0

[277]	valid_0's rmse: 140.418
[278]	valid_0's rmse: 139.808
[279]	valid_0's rmse: 139.15
[280]	valid_0's rmse: 138.544
[281]	valid_0's rmse: 137.866
[282]	valid_0's rmse: 137.318
[283]	valid_0's rmse: 136.8
[284]	valid_0's rmse: 136.267
[285]	valid_0's rmse: 135.671
[286]	valid_0's rmse: 135.073
[287]	valid_0's rmse: 134.488
[288]	valid_0's rmse: 133.936
[289]	valid_0's rmse: 133.387
[290]	valid_0's rmse: 132.99
[291]	valid_0's rmse: 132.5
[292]	valid_0's rmse: 131.934
[293]	valid_0's rmse: 131.338
[294]	valid_0's rmse: 130.882
[295]	valid_0's rmse: 130.48
[296]	valid_0's rmse: 130.046
[297]	valid_0's rmse: 129.639
[298]	valid_0's rmse: 129.213
[299]	valid_0's rmse: 128.866
[300]	valid_0's rmse: 128.458
[301]	valid_0's rmse: 128.101
[302]	valid_0's rmse: 127.711
[303]	valid_0's rmse: 127.311
[304]	valid_0's rmse: 126.969
[305]	valid_0's rmse: 126.548
[306]	valid_0's rmse: 126.212
[307]	valid_0's rmse: 125.86
[308]	valid_0's rmse: 125.561
[309]	valid_0's rmse: 125.164
[310]	valid_0's rm

[563]	valid_0's rmse: 90.3346
[564]	valid_0's rmse: 90.2485
[565]	valid_0's rmse: 90.1949
[566]	valid_0's rmse: 90.1248
[567]	valid_0's rmse: 90.0095
[568]	valid_0's rmse: 89.9493
[569]	valid_0's rmse: 89.9053
[570]	valid_0's rmse: 89.8299
[571]	valid_0's rmse: 89.7772
[572]	valid_0's rmse: 89.7319
[573]	valid_0's rmse: 89.6823
[574]	valid_0's rmse: 89.639
[575]	valid_0's rmse: 89.5982
[576]	valid_0's rmse: 89.5463
[577]	valid_0's rmse: 89.5286
[578]	valid_0's rmse: 89.4538
[579]	valid_0's rmse: 89.4126
[580]	valid_0's rmse: 89.377
[581]	valid_0's rmse: 89.333
[582]	valid_0's rmse: 89.2541
[583]	valid_0's rmse: 89.2109
[584]	valid_0's rmse: 89.1609
[585]	valid_0's rmse: 89.1121
[586]	valid_0's rmse: 89.0357
[587]	valid_0's rmse: 88.9886
[588]	valid_0's rmse: 88.9489
[589]	valid_0's rmse: 88.9092
[590]	valid_0's rmse: 88.829
[591]	valid_0's rmse: 88.755
[592]	valid_0's rmse: 88.6793
[593]	valid_0's rmse: 88.6481
[594]	valid_0's rmse: 88.5874
[595]	valid_0's rmse: 88.5228
[596]	valid_0's

[840]	valid_0's rmse: 80.729
[841]	valid_0's rmse: 80.711
[842]	valid_0's rmse: 80.7031
[843]	valid_0's rmse: 80.6715
[844]	valid_0's rmse: 80.6498
[845]	valid_0's rmse: 80.6381
[846]	valid_0's rmse: 80.617
[847]	valid_0's rmse: 80.5926
[848]	valid_0's rmse: 80.5689
[849]	valid_0's rmse: 80.5603
[850]	valid_0's rmse: 80.5355
[851]	valid_0's rmse: 80.5193
[852]	valid_0's rmse: 80.515
[853]	valid_0's rmse: 80.5
[854]	valid_0's rmse: 80.4733
[855]	valid_0's rmse: 80.4644
[856]	valid_0's rmse: 80.4445
[857]	valid_0's rmse: 80.4286
[858]	valid_0's rmse: 80.4106
[859]	valid_0's rmse: 80.3941
[860]	valid_0's rmse: 80.3812
[861]	valid_0's rmse: 80.3643
[862]	valid_0's rmse: 80.3377
[863]	valid_0's rmse: 80.3332
[864]	valid_0's rmse: 80.3261
[865]	valid_0's rmse: 80.299
[866]	valid_0's rmse: 80.2808
[867]	valid_0's rmse: 80.2499
[868]	valid_0's rmse: 80.2326
[869]	valid_0's rmse: 80.223
[870]	valid_0's rmse: 80.2121
[871]	valid_0's rmse: 80.1872
[872]	valid_0's rmse: 80.1801
[873]	valid_0's rms

In [6]:
print test_set_pre.shape
pre = m.predict(test_set_pre)
print pre
print org[-10:,misscolidx]

(10, 59)
[3016.07865625 2995.10821875 2981.98121875 2969.00540625 2960.70628125
 2949.569875   2945.0098125  2944.11915625 2947.92053125 4252.42178125]
[2983.75    2974.84375 2965.9375  2957.03125 2948.125   2939.21875
 2930.3125  2921.40625 2912.5     2903.59375]


In [35]:
import random
r = random.sample(range(100),5)

In [11]:
np.linalg.norm(res1 - org[-10:,misscolidx]) 

822.9341630404211

In [7]:
res2 = pre #5000 3000
np.linalg.norm(res2 - org[-10:,misscolidx]) 

205.69489300545845

In [7]:
res3 = pre
np.linalg.norm(res3 - org[-10:,misscolidx]) 

1350.350164027599